<a href="https://colab.research.google.com/github/gmihaila/machine_learning_things/blob/master/learning_pytorch/pytorch_simple_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installs

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://github.com/ThilinaRajapakse/simpletransformers">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

In [0]:
from IPython.display import clear_output

!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

%cd ..

!pip install git+https://github.com/explosion/spacy-transformers.git
!pip install git+https://github.com/huggingface/transformers.git

!pip install seqeval

!pip install tensorboardx

# !pip install simpletransformers
!pip install git+https://github.com/ThilinaRajapakse/simpletransformers.git

clear_output()

### Imports

In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
## Load TF 2.0
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import torch
import pandas as pd
import sklearn

from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import MultiLabelClassificationModel


TensorFlow 2.x selected.


### Binary Classification

In [0]:
!rm -r cache_dir  outputs  runs

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_data = [['Example sentence belonging to class 1', 1], ['Example sentence belonging to class 0', 0]]
train_df = pd.DataFrame(train_data)

eval_data = [['Example eval sentence belonging to class 1', 1], ['Example eval sentence belonging to class 0', 0]]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
# model = ClassificationModel('roberta', 'roberta-base', use_cuda=torch.cuda.is_available())

model = ClassificationModel('xlnet', 'xlnet-base-cased', use_cuda=True) 
# You can set class weights by using the optional weight argument


# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)
print(result)
print(model_outputs)


# Predicitons
predictions, raw_outputs = model.predict(['This thing is entirely different from the other thing. '])
print(predictions)
print(raw_outputs)

Converting to features started.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.753642Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:91: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Training of xlnet model complete. Saved to outputs/.
Converting to features started.



{'mcc': 0.0, 'tp': 0, 'tn': 1, 'fp': 0, 'fn': 1, 'acc': 0.5, 'eval_loss': 0.7035998106002808}
[[ 0.29541016 -0.03393555]
 [ 0.28955078 -0.05441284]]
Converting to features started.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)



[1]
[[0.11486816 0.58154297]]


### Multi Class

In [0]:
!rm -r cache_dir  outputs  runs

# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns, a 'text' and a 'labels' column. The `labels` column should contain multi-hot encoded lists.
train_data = [['Example sentence 1 for multilabel classification.', [1, 1, 1, 1, 0, 1]]] + [['This is another example sentence. ', [0, 1, 1, 0, 0, 0]]]
train_df = pd.DataFrame(train_data, columns=['text', 'labels'])
train_df = pd.DataFrame(train_data)

eval_data = [['Example eval sentence for multilabel classification.', [1, 1, 1, 1, 0, 1]], ['Another example eval sentence.', [0, 1, 0, 0, 0, 0]], ['Example eval senntence belonging to class 2', [0, 1, 1, 0, 0, 0]]]
eval_df = pd.DataFrame(eval_data)

print(train_df)
print(eval_df)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel('roberta', 
                                      'roberta-base', 
                                      num_labels=6,
                                      use_cuda=torch.cuda.is_available(),
                                      args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 5})

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(model_outputs)

# Predicitons
predictions, raw_outputs = model.predict(['This thing is entirely different from the other thing. '])
print(predictions)
print(raw_outputs)

                                                   0                   1
0  Example sentence 1 for multilabel classification.  [1, 1, 1, 1, 0, 1]
1                 This is another example sentence.   [0, 1, 1, 0, 0, 0]
                                                   0                   1
0  Example eval sentence for multilabel classific...  [1, 1, 1, 1, 0, 1]
1                     Another example eval sentence.  [0, 1, 0, 0, 0, 0]
2        Example eval senntence belonging to class 2  [0, 1, 1, 0, 0, 0]
Converting to features started.


Running loss: 0.672183

Running loss: 0.699295

Running loss: 0.678025

Running loss: 0.674366

Running loss: 0.710182
Training of roberta model complete. Saved to outputs/.
Converting to features started.



{'LRAP': 0.9755555555555556, 'eval_loss': 0.6707159876823425}
[[0.50097656 0.53271484 0.50634766 0.4260254  0.49658203 0.44360352]
 [0.49951172 0.5317383  0.50341797 0.4248047  0.49658203 0.44604492]
 [0.49853516 0.53125    0.50341797 0.4350586  0.49658203 0.44995117]]
Converting to features started.



[[1, 1, 0, 0, 0, 0]]
[[0.5024414  0.5332031  0.49902344 0.42358398 0.4975586  0.4428711 ]]
